# Data Splitting and Modelling

Author: Gillian A. McGinnis, final-semester M.S. Information Science - Machine Learning  
The University of Arizona College of Information  
INFO 698 - Capstone  
Start date: 21 October 2025  
Last updated: 29 November 2025

*Please note: using a device/service with a **GPU is highly recommended** for running this script.
In fact, not all functions may run as expected if using a CPU (due to minor adjustments for `cudf`, `cupy`, and `cuml`).

The notebook runtime on an `NVIDIA A100-SXM4-40GB` environment is.

In [1]:
"""
Module providing code for test/train split and sliding window creation. Relies on 01_clean.ipynb completion.
"""

'\nModule providing code for test/train split and sliding window creation. Relies on 01_clean.ipynb completion.\n'

## Setup

### Packages

In [2]:
# GPU Setup
%load_ext cudf.pandas
import pandas as pd
import cudf
import cupy as cp

import cuml.accel
cuml.accel.install()

# General packages
import numpy as np
from scipy.stats import randint, uniform

# For making model
import xgboost as xgb

# For getting medians for windowing/smoothing
from scipy.signal import medfilt

# For saving models
import joblib
import pickle

# For metrics and stats
from sklearn.model_selection import TimeSeriesSplit, train_test_split, RandomizedSearchCV, TunedThresholdClassifierCV
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, precision_recall_curve, make_scorer, roc_auc_score, auc

# For help with model tuning
from sklearn.base import clone

In [62]:
# For cloning repo to Colab

from google.colab import userdata
import os

if os.getcwd() == '/content':
  gh_repo = userdata.get('gh_repo')
  if not os.path.exists(f'{gh_repo}'):
    print("Cloning repo...")
    gh_pat = userdata.get('gh_pat')
    gh_user = userdata.get('gh_user')
    repo_url = f'https://{gh_pat}@github.com/{gh_user}/{gh_repo}'
    !git clone {repo_url}
    del gh_pat, gh_user, repo_url
  print("Changing wd...")
  os.chdir(f'{gh_repo}/code')
  del gh_repo

# # Verify the current working directory
print(f"Current working directory is: {os.getcwd()}")


Cloning repo...
Cloning into 'info-698-capstone'...
remote: Enumerating objects: 524, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 524 (delta 137), reused 55 (delta 31), pack-reused 328 (from 1)
Receiving objects: 100% (524/524), 43.26 MiB | 44.57 MiB/s, done.
Resolving deltas: 100% (313/313), done.
Changing wd...
Current working directory is: /content/info-698-capstone/code


In [63]:
# For data importing and exporting
# Note: internal functions from repo
from helper_utils import get_path, model_path, apply_model, report_scores

In [5]:
# # To make it easier to tell when processes have completed -- can delete later
# Google colab compatible:
# https://stackoverflow.com/a/68582785/23486987
from IPython.display import Audio, display

def play_chime():
  return Audio(get_path('completed.mp3', 'code'), autoplay=True)

In [6]:
## (Optional chunk)
# Current session information

# From StackOverflow,
# https://stackoverflow.com/a/62128239/23486987
try:
    import session_info
except:
    !pip install session_info
    import session_info

session_info.show(dependencies=False)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 3.8 MB/s eta 0:00:00


In [7]:
# Make sure GPU active
## !nvidia-smi
import torch
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print(
        "Heads up: GPU not found!\n",
        "Some notebook functionality may be unavailable."
      )

Using GPU: NVIDIA A100-SXM4-40GB


## Inputs

### Constants

In [8]:
## Variable of interest
# for the model to predict
VAR_OF_INTEREST = "obstruction_ro"

## Date subsets
# (for testing smaller runs)
# Set to None to use all data
DATE_START = None
DATE_END = None
# DATE_START = '2001-02-01 00:00:00'
# DATE_END = '2011-12-31 23:59:59'

## Cutoff for correlation for feature removal
# (i.e., columns that are at least this correlated will be removed)
CORR_CUTOFF = 0.97
# File name if the list already exists
CORR_LIST = 'to_drop.pkl'

## Number of splits
# for hyperparameter tuning via expanding window
N_SPLITS = 5

## Number of rounds after which to stop the build
# once the performance stops improving
EARLY_STOPPING_ROUNDS = 50

# Define hyperparameter distributions for random search
N_ESTIMATORS = 1000
# N_ESTIMATORS = 100

## Number of times to iterate the tuning loop
# (set to a small number for testing)
N_ITERATIONS = 50
# N_ITERATIONS = 5


## Smoothing window sizes -- must be odd
# Smallest window size
WINDOW_MIN = 1
# Largest window size
WINDOW_MAX = 35
# WINDOW_MAX = 10

## Model names
MODEL_NAME = "final_full_mod"
FITTED_MODEL_NAME = MODEL_NAME+"_fitted"

## Seed for reproducability
SEED = 42

In [9]:
# Set seed
np.random.seed(SEED)
cp.random.seed(SEED)

### Data

In [10]:
united_water = pd.read_parquet(get_path('clean/water_nocal.parquet'))
united_soil = pd.read_parquet(get_path('clean/soil.parquet'))

data_cal = pd.read_parquet(get_path('clean/calibration.parquet'))
data_cal = data_cal.rename(columns={'weir_level':'weir_level_cal'})

#### Memory improvements

Small amount of data wrangling for memory improvements (some as a consequence of importing).

In [11]:
# Select columns of interest
data_water = united_water.drop(columns=['raw_rain', 'chk_note_rain', 'chk_fail_rain', 'chk_note_ro', 'chk_fail_ro', 'comment_ro', 'source_ro'])

# Cleanup
del united_water

# Remove duplicate entries
data_water = data_water.reset_index().drop_duplicates(keep='first').set_index('datetime')

In [12]:
water_drops = ['level_ro', 'obstruction_ro', 'gap_fill_ro', 'weir_cleaning_ro', 'spike_ro', 'calibration_ro']
water_drops.remove(VAR_OF_INTEREST)

data_water = data_water.drop(water_drops, axis=1)

del water_drops

In [13]:
united_soil['sample'] = united_soil['sample'].astype('category')

In [14]:
# Subset data (if applicable)
data_water = data_water[DATE_START:DATE_END]

In [15]:
# y_all = data_water[VAR_OF_INTEREST]
# data_water = data_water.drop(VAR_OF_INTEREST, axis=1)

## Feature Engineering

XGBoost only looks at one set of inputs at a time. Therefore, it is critical that a representative set of variables are selected or created ("engineered") to make the inputs representative of recent historic data points.

### Distance from Event

In [16]:
def timesince_feat(input_df, input_col, input_unit):
    """Get the time since the occurance of a feature.

    Args:
        input_df (np.ndarray or cp.ndarray): Array of interest.
        input_col (str): Column to make the features from.
        input_unit (str): Granularity to report, by minutes or days.

    Returns:
        output_df: The input_df with added decay columns.
    """
    output_df = input_df
    instances = output_df[input_col].notna()
    # Create groupings based on most recent instance
    group_id = instances.cumsum()
    # Exclude the first grouping
    # otherwise it assumes there was an event just prior to the first entry
    group_id = group_id.replace(0, np.nan)
    # Create new column to count the distance in days since the point
    # which resets to 0 at each new point
    output_df['timestamp'] = pd.to_datetime(output_df.index)
    # Get start timestamp of the group
    output_df['ts_start'] = output_df.groupby(group_id)['timestamp'].transform('min')
    # Calculate the distance
    if input_unit == "minutes":
        output_df[f"minsince_{input_col}"] = (output_df['timestamp'] - output_df['ts_start']).dt.total_seconds().div(60).astype(np.float32)
    elif input_unit == "days":
        output_df[f"daysince_{input_col}"] = (output_df['timestamp'] - output_df['ts_start']).dt.days.astype(np.float32)
    # Remove extra cols
    output_df = output_df.drop(columns=['timestamp', 'ts_start'])
    return output_df

#### Rain
Create feature which tracks how recent a rain event occurred.

In [17]:
data_water = timesince_feat(data_water, 'ra_rain', "minutes")

### Rain event

Keep track of cumulative rainfall during a specific event.

In [18]:
# Create index of instances where there is a data point
rain_event = (data_water['ra_rain'].isnull() & ((data_water['minsince_ra_rain'] >= 5.0) & (data_water['minsince_ra_rain'] != 0)))
# Create groupings based on most recent instance
rain_event_id = rain_event.cumsum()
# Create new column to count number of records since the point
# which resets to 0 at each new point
data_water['eventsum_ra_rain'] = data_water.groupby(rain_event_id)['ra_rain'].cumsum()

del rain_event, rain_event_id

### Decay

In [19]:
def decay_feat(input_df, input_col, input_dec_rate = -0.1):
    """Get rate-decaying feature, and forward fill.

    Args:
        input_df (np.ndarray or cp.ndarray): Array of interest.
        input_col (str): Column to make the features from.
        input_dec_rate (float): Decay rate.

    Returns:
        output_df: The input_df with added decay columns.
    """
    output_df = input_df
    # Add the temporal distance column if not already present.
    if f"minsince_{input_col}" not in output_df.columns:
        output_df = timesince_feat(input_df = output_df, input_col = input_col, input_unit = "minutes")

    # Update for GPU for overflow fix
    output_df[f"minsince_{input_col}"] = output_df[f"minsince_{input_col}"].astype(np.float64)
    # Add column for decay rate
    output_df[f"decayrate{input_dec_rate}_{input_col}"] = np.exp(input_dec_rate * output_df[f"minsince_{input_col}"]).astype(np.float32)
    # Forward fill the feature
    output_df[f"ffill_{input_col}"] = output_df[input_col].ffill()
    # Calculate decay
    output_df[f"decay{input_dec_rate}_{input_col}"] = (output_df[f"ffill_{input_col}"] * output_df[f"decayrate{input_dec_rate}_{input_col}"])

    return output_df

In [20]:
# Replace NAs in rain with 0
data_water['ra_rain'] = data_water['ra_rain'].fillna(0)

# Apply decay function
data_water = decay_feat(data_water, 'eventsum_ra_rain')

# Drop extra column
# (minutes since rain event will be the same as minutes since most recent rain)
data_water = data_water.drop('minsince_eventsum_ra_rain', axis=1)

#### Aside: column consistency

Modify the rows to prevent inappropriate data shifts.
This expands the index to include _all_ possible 5 minute time stamps, so that lagging by index difference guarantees the temporal consistency of lagged features.
This is important because there are some gaps of data, and it would be incorrect for a features that measures "rainfall 5 minutes ago" was actually representing that of the immediately previous point a few hours (or, in some cases, _years_) back due to large data gaps.

In [21]:
original_indices = data_water.index.copy()

new_index = pd.date_range(start = data_water.index.min(),
                          end = data_water.index.max(),
                          freq = '5min')

# Reindex
data_water = data_water.reindex(new_index)

# Cleanup
del new_index

### Rolling stats

Similarly to lag features, rolling statistics can help XGBoost determine how typical recent behavior is, or any patterns that may emerge with "abnormal" or extreme behavior (such as a sharp slope).

In [22]:
def rolling_feats(input_df, input_cols, input_windows, input_mtype = "mean"):
    output_df = input_df

    # Create dummy series of index values (0, 1, 2, ... N)
    # x represents the position within the df for the regression calculation
    x_series = pd.Series(np.arange(len(output_df)), index=output_df.index)

    for col in input_cols:
        for window in input_windows:
            # Calculate general stats
            if input_mtype == "mean":
                output_df[f"{col}_rollmean_{window}"] = output_df[col].rolling(window).mean().astype(np.float32)
            elif input_mtype == "sum":
                output_df[f"{col}_rollsum_{window}"] = output_df[col].rolling(window).sum().astype(np.float32)
            elif input_mtype == "both":
                output_df[f"{col}_rollmean_{window}"] = output_df[col].rolling(window).mean().astype(np.float32)
                output_df[f"{col}_rollsum_{window}"] = output_df[col].rolling(window).sum().astype(np.float32)
            output_df[f"{col}_rollstd_{window}"] = output_df[col].rolling(window).std().astype(np.float32)

            # Calculating slope w vecotrized options
            # Calculate covariance of y (data) vs X (index)
            rolling_cov = output_df[col].rolling(window).cov(x_series)
            # Calculate variance of X (index)
            rolling_var_x = x_series.rolling(window).var()
            # Slope = Cov(Y, X) / Var(X)
            output_df[f"{col}_rollslope_{window}"] = (rolling_cov / rolling_var_x).astype(np.float32)
    return output_df

In [23]:
# Inclusive of current point--
# # 10m, 15m, 20m, 25m, 30m, 1h, 3h, 6h, 12h, 24h
# windows_of_interest = [2, 3, 4, 5, 6, 12, 36, 72, 144, 288]
# # 10m, 15m, 30m, 1h, 6h, 12h, 24h
windows_of_interest = [2, 3, 6, 12, 72, 144, 288]
data_water = rolling_feats(data_water, ['raw_ro'], windows_of_interest, "sum")

# 10m, 30m, 1h, 3h, 6h
# windows_of_interest = [2, 3, 4, 5, 6, 12, 36, 72, 144, 288]
windows_of_interest = [2, 6, 12, 36, 72]
data_water = rolling_feats(data_water, ['ra_rain'], windows_of_interest, "sum")

Change since last value

In [24]:
data_water['raw_ro_change'] = data_water['raw_ro'].diff()
data_water['ra_rain_change'] = data_water['ra_rain'].diff()

# data_water['raw_ro_rollmean_2_change'] = data_water['raw_ro_rollmean_2'].diff()

### Lag features

Because XGBoost predicts on one data point at a time, time series data must include "lag" features (i.e., data points prior to that event) in order to better repsent recent history.

Get values from other recent time stamps.

In [25]:
def lag_feats(input_df, input_cols, input_lags):
    """Get lag features of interest.

      Args:
          input_df (np.ndarray or cp.ndarray): Array of interest.
          input_col (str): Column to make the features from.
          input_lags (list): Past values to pull from, in 5 min intervals.

      Returns:
          output_df: The input_df with added decay columns.
    """
    output_df = input_df#.copy()
    for col in input_cols:
        for lag in input_lags:
            output_df[f"{col}_lag{lag}"] = output_df[col].shift(lag)
    return output_df

In [26]:
# Columns to get temporal stats on
# cols_to_shift = ['raw_ro', 'ra_rain']
# cols_to_shift = ['raw_ro']

# data at 5-min increments -- lag to record values at 5m, 10m, 15m, 20m, 25m, 30m, 1h, 2h, 3h prior
# lags_of_interest = [1, 2, 3, 4, 5, 6, 12, 24, 36]
# lags_of_interest = range(1, 37+1, 2)

# data at 5-min increments -- lag to record values at every 10 min from 5m-2h prior
data_water = lag_feats(data_water, ['raw_ro'], range(1, 24+1, 2))

cols_rainsum = [col for col in data_water.columns if col.startswith('ra_rain_rollsum_')]
# 15m, 1h, 3h
# lags_of_interest = [3, 12, 36]

for col in cols_rainsum:
    data_water = lag_feats(data_water, [col], [3, 12, 36])

# data_water.sample(10)

In [27]:
# # Revert index
# (CPU method)
# data_water = data_water.loc[original_indices]

# del original_indices

In [28]:
# Revert index
# (adjusted for GPU)

data_water_reset = data_water.reset_index()
index_col_name = data_water_reset.columns[0]
indices_df = original_indices.to_frame(name=index_col_name)

filtered_data_water = cudf.merge(
    data_water_reset,
    indices_df,
    on=index_col_name,
    how='inner'
)
data_water = filtered_data_water.set_index(index_col_name)

del original_indices, filtered_data_water, index_col_name, data_water_reset, indices_df

## Soil

Pivot the soil data such that each sample has its own columns, and separated by depth.

In [29]:
# Drop irrelevant column
data_soil_shallow = united_soil.copy().drop('h2o_by_wet_deep', axis=1)
data_soil_shallow['sample'] = data_soil_shallow['sample'].astype('float32')
# Pivot wider
data_soil_shallow = data_soil_shallow.pivot(columns='sample', values='h2o_by_wet_shallow')

# Drop irrelevant column
data_soil_deep = united_soil.copy().drop('h2o_by_wet_shallow', axis=1)

data_soil_deep['sample'] = data_soil_deep['sample'].astype('float32')
# Pivot wider
data_soil_deep = data_soil_deep.pivot(columns='sample', values='h2o_by_wet_deep')

del united_soil

In [30]:
data_soil = pd.merge(
    data_soil_shallow,
    data_soil_deep,
    left_index = True,
    right_index = True,
    suffixes = ("_shallow", "_deep"),
    how = "outer"
)

del data_soil_shallow, data_soil_deep

## Unite

Join the data frames to prep for the model.

In [31]:
data_united = pd.merge(
    data_water,
    data_cal[DATE_START:DATE_END],
    left_index = True,
    right_index = True,
    how = 'outer'
)

del data_water, data_cal

data_united = pd.merge(
    data_united,
    data_soil[DATE_START:DATE_END],
    left_index = True,
    right_index = True,
    how = 'outer'
)

del data_soil

### United features

Add a few final features on the united data set.

In [32]:
# Difference compared to calibration point (infrequent)
data_united['diff_ro_cal'] = (data_united['weir_level_cal'] - data_united['raw_ro'])

# Convert to float
data_united['diff_ro_cal'] = data_united['diff_ro_cal'].astype(np.float32)

# Time since last calibration point
data_united = timesince_feat(data_united, 'weir_level_cal', "minutes")

In [33]:
# Create features to track soil value staleness
cols_soil = [col for col in data_united.columns if (col.endswith('shallow') | col.endswith('deep'))]

for col in cols_soil:
    data_united = timesince_feat(data_united, col, "days")

# Extend soil vals
data_united[cols_soil] = data_united[cols_soil].ffill()

del col, cols_soil

### Temporal features
Adding temporal features to the data can help with seasonality or time-of-day considerations.

This will modify temporal features to be based on sine and cosine transformations, which allows for the model to be based on the cyclical patterns of time rather than abrupt distances (e.g., the numeric value Day 365 is 'far' from Day 001, but in reality they are very near).

In [34]:
def temporal_feat(input_df, input_unit):
  output_df = input_df
  # Day of the year
  if input_unit=='day':
    cycle_length = 365.25
    value = output_df.index.dayofyear
  # Month of the year
  # (largest granularity)
  elif input_unit=='month':
    cycle_length = 12
    value = output_df.index.month
  # Hour of the day
  elif input_unit=='hour':
    cycle_length = 24
    value = output_df.index.hour
  # Minute on the hour
  # (finest granularity)
  elif input_unit=='minute':
    cycle_length = 60
    value = output_df.index.minute

  output_df[f'{input_unit}_sin'] = np.sin(2 * np.pi * value / cycle_length).astype(np.float32)
  output_df[f'{input_unit}_cos'] = np.cos(2 * np.pi * value / cycle_length).astype(np.float32)

  return output_df

In [35]:
data_united = temporal_feat(data_united, 'minute')
data_united = temporal_feat(data_united, 'hour')
data_united = temporal_feat(data_united, 'day')
data_united = temporal_feat(data_united, 'month')

## Data split

First, rows with no recorded `y` value are dropped, where `y` is the variable of interest.
The `X` set includes all other features.

In [36]:
# REMOVE NAs
data_united = data_united.dropna(subset=[VAR_OF_INTEREST])

X_all = data_united.drop(VAR_OF_INTEREST, axis=1).copy()
y_all = data_united[VAR_OF_INTEREST].copy()

In [37]:
## Random data type fixes for GPU usage

y_all = y_all.astype(np.float32)
y_all = y_all.as_gpu_object()

for col in X_all.columns:
  if str(X_all[col].dtype) == ('Int32'):
    X_all[col] = X_all[col].astype(np.float32)

### Train/Test split

The test set is to be put aside until the final (fully tuned and fitted) model has been created.

Unlike the typical approach for train/test splits, temporal data in this context must _not_ be randomly split as it would lead to severe leakage.

The ratio will be 80/20 train/test.

In [38]:
# Conduct the split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.2, shuffle=False)

# Cleanup
# especially because the X set is rather memory-intensive
del X_all, y_all

print(
    "Train:\tn =", len(X_train), "\t", X_train.index[0], "thru", X_train.index[-1],
    "\nTest:\tn =", len(X_test), "\t", X_test.index[0], "thru", X_test.index[-1]
)

Train:	n = 2845192 	 1989-07-19 11:55:00 thru 2018-10-27 17:05:00 
Test:	n = 711299 	 2018-10-27 17:10:00 thru 2025-08-01 13:00:00


### Expanding window

<!-- This will conduct an 80/20 initial train/test split, with expanding sliding window for training/validation to tune hyperparameters and test model stability. -->

For tuning hyperparameters and test model stability, an expanding window approach will be used. This is similar to how a model would act once deployed, as it will only gain more data over time.

In [39]:
# Initialize the split function
tscv = TimeSeriesSplit(n_splits=N_SPLITS)

print("------------------------------------------------------------")
for i, (train_index, val_index) in enumerate(tscv.split(X_train)):
    train_i_start = X_train.index[train_index].min()
    train_i_end = X_train.index[train_index].max()
    val_i_start = X_train.index[val_index].min()
    val_i_end = X_train.index[val_index].max()
    print(f"Fold {i+1}:")
    # print(f"\tTrain: index={train_index}")
    print(f"\tTrain:\t{train_i_start} thru {train_i_end}")
    # print(f"\tValidation:  index={val_index}")
    print(f"\tVal:\t{val_i_start} thru {val_i_end}")
    # print("  Train: index=", mini_x.index[train_index])
    # print(f"  Test:  index={val_index}")
    print("------------------------------------------------------------")

del i, train_index, val_index
del train_i_start, train_i_end, val_i_start, val_i_end

------------------------------------------------------------
Fold 1:
	Train:	1989-07-19 11:55:00 thru 1994-02-04 12:30:00
	Val:	1994-02-04 12:35:00 thru 1998-12-01 19:25:00
------------------------------------------------------------
Fold 2:
	Train:	1989-07-19 11:55:00 thru 1998-12-01 19:25:00
	Val:	1998-12-01 19:30:00 thru 2003-06-19 15:40:00
------------------------------------------------------------
Fold 3:
	Train:	1989-07-19 11:55:00 thru 2003-06-19 15:40:00
	Val:	2003-06-19 15:45:00 thru 2008-01-20 14:40:00
------------------------------------------------------------
Fold 4:
	Train:	1989-07-19 11:55:00 thru 2008-01-20 14:40:00
	Val:	2008-01-20 14:45:00 thru 2012-08-07 23:35:00
------------------------------------------------------------
Fold 5:
	Train:	1989-07-19 11:55:00 thru 2012-08-07 23:35:00
	Val:	2012-08-07 23:40:00 thru 2018-10-27 17:05:00
------------------------------------------------------------


## Parameter selection

Highly correlated features can be removed prior to model selection.
So long as the _same_ features are dropped in the test set as well, this will not result in data leakage.

This takes a bit of time, so will load any cached list from a prior run.

In [40]:
# @title
# pickle_path = get_path('booah', 'outputs')

# if os.path.exists(pickle_path) == True:
#   print("Vars to drop previously cached...")
#   with open(pickle_path, 'rb') as file:
#     to_drop = pickle.load(file)
# else:
#   print("Finding corr vars...")

#   # X_train_corrblock = X_train.copy() # Backup

#   # Calculate the correlation matrix for all features
#   # corr = X_train_corrblock.corr().abs()
#   corr_np = np.corrcoef(X_train.values, rowvar=False)

#   # convert to pd
#   corr = pd.DataFrame(corr_np,
#                       columns=X_train.columns,
#                       index=X_train.columns).abs()

#   corr.info()
#   print(corr)

#   # Select only half of the correlated vars
#   # e.g., if A and B are highly correlated, only select A to drop
#   upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

#   # Get the list of features to drop
#   to_drop = [col for col in upper.columns if any(upper[col] > CORR_CUTOFF)]

#   # Save the list
#   with open(pickle_path, "wb") as fp:
#     pickle.dump(to_drop, fp)
#   # del X_train_corrblock

# del pickle_path

# print(f"{len(to_drop)} correlated features (correlation >{CORR_CUTOFF}) dropped:\n",
#     to_drop)

In [41]:
pickle_path = get_path(CORR_LIST, 'outputs')

if os.path.exists(pickle_path) == True:
  print("Vars to drop previously cached...")
  with open(pickle_path, 'rb') as file:
    to_drop = pickle.load(file)
else:
  print("Finding corr vars...")

  # X_train_corrblock = X_train.copy() # Backup

  # Calculate the correlation matrix for all features
  corr = X_train.corr().abs()
  # corr = np.corrcoef(X_train_corrblock.values, rowvar=False)

  # Select only half of the correlated vars
  # e.g., if A and B are highly correlated, only select A to drop
  upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

  # Get the list of features to drop
  to_drop = [col for col in upper.columns if any(upper[col] > CORR_CUTOFF)]

  # Save the list
  with open(pickle_path, "wb") as fp:
    pickle.dump(to_drop, fp)
  # del X_train_corrblock

del pickle_path

print(f"{len(to_drop)} correlated features (correlation >{CORR_CUTOFF}) dropped:\n",
    to_drop)

Finding corr vars...
29 correlated features (correlation >0.97) dropped:
 ['ffill_eventsum_ra_rain', 'decay-0.1_eventsum_ra_rain', 'raw_ro_rollsum_2', 'raw_ro_rollsum_3', 'raw_ro_rollsum_6', 'raw_ro_rollsum_12', 'raw_ro_change', 'ra_rain_change', 'raw_ro_lag1', 'raw_ro_lag3', 'raw_ro_lag5', 'raw_ro_lag7', 'ra_rain_rollsum_36_lag3', 'ra_rain_rollsum_72_lag3', 'daysince_2.0_shallow', 'daysince_4.0_shallow', 'daysince_5.0_shallow', 'daysince_9.0_shallow', 'daysince_10.0_shallow', 'daysince_1.0_deep', 'daysince_2.0_deep', 'daysince_3.0_deep', 'daysince_4.0_deep', 'daysince_5.0_deep', 'daysince_6.0_deep', 'daysince_7.0_deep', 'daysince_8.0_deep', 'daysince_9.0_deep', 'daysince_10.0_deep']


In [42]:
# # Backups
# X_train_backup = X_train.copy()
# X_test_backup = X_test.copy()

# Remove the columns
X_train = X_train.drop(columns=to_drop)
X_test = X_test.drop(columns=to_drop)

## Hyperparameter tuning

As per XGBoosting documentation/tutorials, early stopping with random search for hyperparameter tuning must be iterated upon manually, as `RandomizedSearchCV` does not support using a separate validation set within each CV fold.

Source: https://xgboosting.com/xgboost-early-stopping-with-random-search/


The area under the precision-recall curve (AUC-PR) can be used to evaluate the performance, since it considers a range of classification thresholds.
This is better than an ROC AUC metric since there is greater class imbalance (i.e., `True` is more rare).

Source: https://xgboosting.com/evaluate-xgboost-performance-with-precision-recall-curve/


In [43]:
# Code modified from
# https://xgboosting.com/xgboost-early-stopping-with-random-search/

# Define hyperparameter distributions for random search
param_distributions = {
    'learning_rate': ('uniform', 0.01, 0.3),
    'max_depth': ('choice', [2, 3, 4, 5, 6]),
    'subsample': ('uniform', 0.5, 1.0),
    'colsample_bytree': ('uniform', 0.4, 1.0),
    'scale_pos_weight':('choice', [5, 7, 9, 10, 11]),
    'gamma': ('uniform', 0, 0.5),
    'reg_alpha': ('uniform', 0, 1.0)
}

# Define seed again
rng = np.random.default_rng(SEED)

# Function to sample parameters based on their distribution type
def sample_param(distribution):
    if distribution[0] == 'uniform':
        # Use seeded generator to create the scipy distribution
        return uniform(loc=distribution[1], scale=distribution[2] - distribution[1]).rvs(random_state=rng)
    elif distribution[0] == 'choice':
        # Use seeded generator choice method
        return rng.choice(distribution[1])
    else:
        raise ValueError(f"Unsupported distribution type: {distribution[0]}")

# Initialize baselines
best_params = None
best_score = 0
best_avg_rounds = 0

# Iterate a number of times
for _ in range(N_ITERATIONS):
  test_scores = []
  best_rounds = []
  optimal_rounds_list = []
  params = {k: sample_param(v) for k, v in param_distributions.items()}

  for train_index, test_index in tscv.split(X_train):
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

    # Split training set again to make an internal validation set for early stopping
    X_train_fold, X_val, y_train_fold, y_val = train_test_split(X_train_fold, y_train_fold, test_size=0.2, shuffle=False)

    # Prepare the model
    model = xgb.XGBClassifier(
        n_estimators=N_ESTIMATORS,
        learning_rate=params['learning_rate'],
        max_depth=int(params['max_depth']),  # needs to be an int
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        objective='binary:logistic',
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        scale_pos_weight=params['scale_pos_weight'],
        ## SETTINGS FOR GPU
        seed_per_iteration = True,
        tree_method='hist',
        device='cuda',
        # Set specific eval metric: AUC-PR
        eval_metric='aucpr',
        random_state=SEED,
        n_jobs=-1,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS # best will be determined in the loop
    )

    # Fit model on train fold and use validation for early stopping
    model.fit(X_train_fold, y_train_fold, eval_set=[(X_val, y_val)], verbose=False)

    # Find optimal number of iterations
    optimal_rounds_list.append(model.best_iteration)

    #### Predict on test set

    ###
    ## Using F1
    # y_pred_test = model.predict(X_test_fold)
    # # test_score = accuracy_score(y_test_fold, y_pred_test)
    # # test_score = f1_score(y_test_fold, y_pred_test)
    # test_score = f1_score(y_test_fold.to_cupy().get(), y_pred_test)
    # test_scores.append(test_score)
    ###
    ## Using ROC AUC
    # y_pred_test = model.predict_proba(X_test_fold)[:,1]
    # # y_pred_test = y_pred_test.to_cupy()
    # # y_pred_test = cp.array(y_pred_test)
    # y_pred_test = cudf.Series(y_pred_test)
    # # test_score = roc_auc_score(y_test_fold, y_pred_test)
    # test_score = cuml.metrics.roc_auc_score(y_test_fold, y_pred_test)
    # test_scores.append(test_score)
    ###

    ## Using AUC PR
    y_pred_test = model.predict_proba(X_test_fold)[:,1]
    # Getting precision and recall
    # with some cupy strangeness
    y_prec, y_rec, _ = precision_recall_curve(y_test_fold.to_cupy().get(), y_pred_test)
    test_score = auc(y_rec, y_prec)
    test_scores.append(test_score)
    ##

  # Compute average score across all folds
  average_score = np.mean(test_scores)
  average_optimal_rounds = np.mean(optimal_rounds_list)

  # If the average score has improved, store the updateed values
  if average_score > best_score:
    best_score = average_score
    best_params = params
    best_avg_rounds = int(round(average_optimal_rounds))


print(f"CV averaged score:\t{average_score:.4f}")
print(f"Best av rounds (n_est):\t{best_avg_rounds}")
print("Best parameters----------")
# aligned printing code from stack overflow,
# https://stackoverflow.com/a/54573735/23486987
for key, value in best_params.items():
    print(f'--- {key:20}{round(value, 4)}')


CV averaged score:	0.2027
Best av rounds (n_est):	193
Best parameters----------
--- learning_rate       0.102
--- max_depth           3
--- subsample           0.6455
--- colsample_bytree    0.709
--- scale_pos_weight    11
--- gamma               0.128
--- reg_alpha           0.936


In [44]:
play_chime()

In [45]:
# Cleanup
del X_train_fold, X_val, y_train_fold, y_val
del y_pred_test
del test_scores, best_rounds, optimal_rounds_list
del average_score, average_optimal_rounds
del y_prec, y_rec

## Save model

In [46]:
if os.path.exists(model_path(MODEL_NAME)) == False:
    print("Creating final model...")

    final_model = xgb.XGBClassifier(
        n_estimators=best_avg_rounds, # avg optimal n_estimators calculated before
        # n_estimators=100, # or reasonable default
        random_state=SEED,
        learning_rate=best_params['learning_rate'],
        max_depth=int(best_params['max_depth']),
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        objective='binary:logistic',
        scale_pos_weight=best_params['scale_pos_weight'],
        gamma=best_params['gamma'],
        reg_alpha=best_params['reg_alpha'],
        ## SETTINGS FOR GPU
        seed_per_iteration = True,
        tree_method='hist',
        device='cuda',
        ##
        n_jobs=-1
    )

    joblib.dump(final_model, model_path(MODEL_NAME))
    # Will not be fitted until after OOF tuning has been conducted
    # (model will be saved separately)

    # Local download
    from google.colab import files
    files.download(model_path(MODEL_NAME))

else:
    print("Importing model from saved files...")
    final_model = joblib.load(model_path(MODEL_NAME))

Creating final model...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## OOF predictions

Out-of-fold (OOF) predictions will be used to tune smoothing and thresholding parameters. To do this, the model with tuned hyperparameters will predict each sliding window set from before. This reflects the real-world performance of the model as it can fit to more data.

In [47]:
oof_pred = np.full(len(y_train), np.nan)

for train_index, test_index in tscv.split(X_train):
  X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
  y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

  # Prepare the model
  model = clone(final_model)

  model.fit(X_train_fold, y_train_fold)
  y_pred_test = model.predict_proba(X_test_fold)[:,1]
  oof_pred[test_index] = y_pred_test

# Isolate the OOF truth values
# because the first training set is not included
valid_mask = ~np.isnan(oof_pred)
y_train_oof = y_train.values[valid_mask]
oof_pred = oof_pred[valid_mask]

del X_train_fold, y_train_fold, X_test_fold, y_test_fold, y_pred_test
del model, valid_mask

### Smoothing & Thresholding

Windowing can help smooth predictions by preventing standalone points that differ from their neighbors (e.g., having a sequence of `True` interrupted by one `False`, or vice-versa, both of which are unlikely in this context due to how weir blockages occur).
Median windowing is often used for image analysis, but is effective at removing outliers/interruptions in the data while keeping edges.

By default, a threshold of 0.5 will be selected for categorizing a point as `True` or `False`. However, in this context a model more sensitive to `True` may make final results more accurate.

These measures can be found by finding the window and corresponding threshold that maximizes the F1 score.

In [48]:
# Fixing more cupy strangeness
def f1_cust(input_true, input_pred):
  """Report the F1 score using inputs that might be mixed type.

    Args:
        input_true (np.ndarray or cp.ndarray): Array of true y values.
        input_parent (np.ndarray or cp.ndarray): Array of predicted y values.

    Returns:
        float: F1 score.
    """
  output_true = input_true.copy()
  output_pred = input_pred.copy()
  if not isinstance(output_true, np.ndarray):
    output_true = output_true.get()
  if not isinstance(output_pred, np.ndarray):
    output_pred = output_pred.get()

  output_f1 = f1_score(output_true, output_pred)
  return output_f1

In [49]:
# Start w no smoothing
best_window_size = 1
# Start w default threshold
threshold = 0.5

y_bin = (oof_pred >= threshold).astype(np.int32)
# y_bin = cp.array(y_bin)
best_f1 = f1_cust(y_train_oof, y_bin)

best_threshold = threshold
thresholds = np.linspace(0.01, 0.99, 100)

# Keep track of tuning results
wt_tune_results = cudf.DataFrame(columns=['window', 'threshold', 'F1'])

print(f"Baseline\t---------->\tW:1\tT:{best_threshold}\t\tF1:{best_f1:.4}")

# Test range of odd window sizes
for current_window in range(WINDOW_MIN, WINDOW_MAX+1, 2):
    smoothed_preds = medfilt(oof_pred, kernel_size=current_window)
    # Test range of possible threshold
    for threshold in thresholds:
        y_bin = (smoothed_preds >= threshold).astype(np.int32)
        current_f1 = f1_cust(y_train_oof, y_bin)
        # Keep track of scores
        wt_tune_result = cudf.Series({"window": current_window, "threshold": threshold, 'F1':current_f1})
        # Add it to the full data frame
        wt_tune_results = cudf.concat([wt_tune_results, wt_tune_result.to_frame().T])
        # If the F1 score has improved, update the values
        if current_f1 > best_f1:
          best_f1 = current_f1
          best_window = current_window
          best_threshold = threshold
    print(f"Trying win={current_window}; \tCurr best->\tW:{best_window}\tT:{best_threshold:.4}\tF1:{best_f1:.4}")

# oof_pred_adj = smooth_window(oof_pred, window_size=best_window)
oof_pred_adj = medfilt(oof_pred, kernel_size=best_window)
oof_pred_adj = (oof_pred_adj >= best_threshold).astype(int)

f1_rez = f1_cust(y_train_oof, oof_pred_adj)
print(f"Best parameters\t---------->\tW:{best_window}\tT:{best_threshold:.4f}\tF1:{f1_rez:.4}")

Baseline	---------->	W:1	T:0.5		F1:0.4317
Trying win=1; 	Curr best->	W:1	T:0.307	F1:0.4534
Trying win=3; 	Curr best->	W:3	T:0.307	F1:0.4538
Trying win=5; 	Curr best->	W:5	T:0.307	F1:0.4541
Trying win=7; 	Curr best->	W:7	T:0.307	F1:0.4542
Trying win=9; 	Curr best->	W:9	T:0.307	F1:0.4545
Trying win=11; 	Curr best->	W:11	T:0.307	F1:0.4548
Trying win=13; 	Curr best->	W:13	T:0.307	F1:0.4549
Trying win=15; 	Curr best->	W:13	T:0.307	F1:0.4549
Trying win=17; 	Curr best->	W:13	T:0.307	F1:0.4549
Trying win=19; 	Curr best->	W:13	T:0.307	F1:0.4549
Trying win=21; 	Curr best->	W:13	T:0.307	F1:0.4549
Trying win=23; 	Curr best->	W:13	T:0.307	F1:0.4549
Trying win=25; 	Curr best->	W:13	T:0.307	F1:0.4549
Trying win=27; 	Curr best->	W:13	T:0.307	F1:0.4549
Trying win=29; 	Curr best->	W:13	T:0.307	F1:0.4549
Trying win=31; 	Curr best->	W:13	T:0.307	F1:0.4549
Trying win=33; 	Curr best->	W:13	T:0.307	F1:0.4549
Trying win=35; 	Curr best->	W:13	T:0.307	F1:0.4549
Best parameters	---------->	W:13	T:0.3070	F1:0.454

In [50]:
play_chime()

### Remove marginal gains

Remembering Occam's Razor... marginal gains in F1 can result in over-fitting (such as applying large smoothing windows for an F1 gain of, for instance, 0.0003).

In [51]:
best_window_original = best_window
best_threshold_original = best_threshold

In [52]:
wt_mg = wt_tune_results.copy()

# Selecting the values where the F1 is within 99% of the best F1
print("Original F1:\t", best_f1)
f1_cutoff = 0.99 * best_f1
print("F1 99% / marginal-gains cutoff:\t", f1_cutoff)

print((len(wt_mg)/len(wt_tune_results)) * 100, "% of tested combinations are w/in 99% of the best F1")
# print(len(wt_mg))

print("-------------------------")

# Arrange in descending order: prioritize smaller windows
best_mg = wt_mg[wt_mg['F1']>=f1_cutoff].sort_values(['window','F1'],ascending=[True,False]).iloc[0]

if best_window != best_mg['window']:
  # print("-------------------------")
  print("New best window found.")
  print("Original:\t", best_window)
  print("Updated:\t", best_mg['window'])
  print("-------------------------")


if best_threshold != best_mg['threshold']:
  # print("-------------------------")
  print("New best threshold found.")
  print("Original:\t", best_threshold)
  print("Updated:\t", best_mg['threshold'])
  print("-------------------------")


# Update values
best_window = int(best_mg['window'])
best_threshold = float(best_mg['threshold'])

if best_window == 1:
  print("F1 did not significantly improve via windowed smoothing.\nNo windowing will be applied.")

Original F1:	 0.45485223437979344
F1 99% / marginal-gains cutoff:	 0.4503037120359955
100.0 % of tested combinations are w/in 99% of the best F1
-------------------------
New best window found.
Original:	 13
Updated:	 1.0
-------------------------
F1 did not significantly improve via windowed smoothing.
No windowing will be applied.


In [53]:
# cleanup
del current_f1, current_window, threshold, f1_rez, y_bin, best_f1, wt_tune_result, oof_pred_adj

## Applying to test set

In [64]:
final_model.fit(X_train, y_train)

y_test_conv = y_test.copy().to_cupy()
y_test_conv = y_test_conv.get()

In [73]:
# Predictions on final test set
# with and without the various best-corrections
pred_y_proba = apply_model(X_test, final_model, "proba", None, None)

pred_y_none, pred_y_w, pred_y_t, pred_y_wt = apply_model(X_test, final_model, "all",
                                                         input_window=best_window,
                                                         input_threshold=best_threshold)

print("Corrections\tF1\tAcc\tPre\tRec")


# Get scores accordingly
if best_window != 1:
  print(
      "--------------------------------------------------",
      "\nNone:\t\t", report_scores(y_test_conv, pred_y_none, 4),
      f"\nW {best_window}:\t\t", report_scores(y_test_conv, pred_y_w, 4),
      f"\nT {best_threshold:.4}:\t", report_scores(y_test_conv, pred_y_t, 4),
      f"\nW{best_window} & T{best_threshold:.4}:\t", report_scores(y_test_conv, pred_y_wt, 4)
  )
else:
  print(
      "--------------------------------------------------",
      "\nNone:\t\t", report_scores(y_test_conv, pred_y_none, 4),
      f"\nT {best_threshold:.4}:\t", report_scores(y_test_conv, pred_y_t, 4)
  )

# Also check scores if the marginal gains filter was not applied
pred_y_none_mg, pred_y_w_mg, pred_y_t_mg, pred_y_wt_mg = apply_model(X_test, final_model, "all",
                                                         input_window=best_window_original,
                                                         input_threshold=best_threshold_original)

print(
    "--------------------------------------------------",
    "\nIf F1 marginal gain limitation were not implemented:",
    "\nNone:\t\t", report_scores(y_test_conv, pred_y_none_mg, 4),
    f"\nW {best_window_original}:\t\t", report_scores(y_test_conv, pred_y_w_mg, 4),
    f"\nT {best_threshold_original:.4}:\t", report_scores(y_test_conv, pred_y_t_mg, 4),
    f"\nW{best_window_original} & T{best_threshold_original:.4}:\t", report_scores(y_test_conv, pred_y_wt_mg, 4)
)

del pred_y_none_mg, pred_y_w_mg, pred_y_t_mg, pred_y_wt_mg

Corrections	F1	Acc	Pre	Rec
-------------------------------------------------- 
None:		 (0.4045, 0.7231, 0.2978, 0.6301) 
T 0.307:	 (0.3682, 0.5798, 0.2373, 0.8205)
-------------------------------------------------- 
If F1 marginal gain limitation were not implemented: 
None:		 (0.4045, 0.7231, 0.2978, 0.6301) 
W 13:		 (0.4052, 0.7242, 0.2987, 0.6298) 
T 0.307:	 (0.3682, 0.5798, 0.2373, 0.8205) 
W13 & T0.307:	 (0.3688, 0.5802, 0.2377, 0.8217)


In [71]:
# Cleanup
del pred_y_none, pred_y_w, pred_y_t, pred_y_wt

## Save results

In [72]:
if os.path.exists(model_path(FITTED_MODEL_NAME)) == False:
  print("Saving final fitted model...")

  joblib.dump(final_model, model_path(FITTED_MODEL_NAME))

  # Local download
  from google.colab import files
  files.download(model_path(FITTED_MODEL_NAME))

Saving final fitted model...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [75]:
## Save to wd

# Tuning results
wt_tune_results.to_parquet(get_path('results/train_oof_wt_tuning.parquet', 'outputs'))

# Indeces of test dates
test_dates = y_test.index.to_frame()
test_dates.to_parquet(get_path('results/test_index.parquet', 'outputs'))

# y predictions
np.save(get_path('results/test_y_pred.npy', 'outputs'), pred_y_wt)
np.save(get_path('results/test_y_pred_proba.npy', 'outputs'), pred_y_proba)

# actual test vals
np.save(get_path('results/test_y_true.npy', 'outputs'), y_test_conv)
# np.save(get_path('results/X_test_true.npy', 'outputs'), X_test)

In [76]:
# Save locally
files.download(get_path('results/train_oof_wt_tuning.parquet', 'outputs'))
files.download(get_path('results/test_index.parquet', 'outputs'))

files.download(get_path('results/test_y_pred.npy', 'outputs'))
files.download(get_path('results/test_y_pred_proba.npy', 'outputs'))

files.download(get_path('results/test_y_true.npy', 'outputs'))
# files.download(get_path('results/X_test_true.npy', 'outputs'))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [77]:
play_chime()